In [1]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 53.2 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-macosx_10_9_universal2.whl size=1610416 sha256=3b30065a57f1eb5c32ade634b069b2a513344b8abc754f4b9ed56bcae820dcae
  Stored in directory: /Users/liu/Library/Caches/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise
--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/Library/Frameworks/Python.framework/Versions/3.10/li

In [2]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import cross_validate

In [3]:
ratings = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/ratings.csv')
movies = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/movies.csv')

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [6]:
data = pd.merge(ratings,movies, on='movieId')

In [7]:
reader = Reader(rating_scale=(0.5,5))

In [8]:
data = Dataset.load_from_df(data[['userId', 'movieId','rating']], reader)

In [9]:
params = {'name':'cosine', 'user_based':False}

In [10]:
model = KNNBasic(sim_options=params)

In [11]:
cross_validate(model, data, measures=['RMSE','MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9748  0.9759  0.9766  0.9701  0.9803  0.9755  0.0033  
MAE (testset)     0.7547  0.7610  0.7614  0.7523  0.7628  0.7584  0.0042  
Fit time          8.53    7.78    7.40    7.79    7.00    7.70    0.51    
Test time         7.62    7.66    8.12    7.09    7.42    7.58    0.34    


{'test_rmse': array([0.97482659, 0.97590563, 0.97655873, 0.97011448, 0.98032049]),
 'test_mae': array([0.75470249, 0.76097353, 0.76138198, 0.75228416, 0.7628442 ]),
 'fit_time': (8.528763055801392,
  7.782315015792847,
  7.396291732788086,
  7.790318727493286,
  6.995657920837402),
 'test_time': (7.6212828159332275,
  7.659411907196045,
  8.119840860366821,
  7.08786416053772,
  7.417799949645996)}

In [12]:
from surprise import accuracy
from surprise.model_selection import train_test_split

In [13]:
trainset, testset =train_test_split(data, test_size=0.2)


In [14]:
model2 = KNNBasic(sim_options={'user_based':True})
model2.fit(trainset)
predictions = model2.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9594


0.9593720408239498

In [15]:
model2 = KNNBasic(min_k = 5, k=10, sim_options={'name':'cosine','user_based':True})
model2.fit(trainset)
predictions = model2.test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9934


0.9933595030973472

In [16]:
model2.predict(668,16)

Prediction(uid=668, iid=16, r_ui=None, est=4.046185871961997, details={'actual_k': 10, 'was_impossible': False})

In [17]:
movies[movies['movieId']==16]

,movieId,title,genres
15,16,Casino (1995),Crime|Drama


In [18]:
final_dataset = pd.merge(movies,ratings, on='movieId')

In [19]:
final_dataset['predicted_rating'] = final_dataset.apply(lambda x:model2.predict(x['userId'],x['movieId']).est, axis=1)

In [20]:
final_dataset.head()

,movieId,title,genres,userId,rating,timestamp,predicted_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,5.0,859046895,4.300000
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1303501039,4.400000
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,5.0,858610933,3.600000
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.0,850815810,4.049455
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14,4.0,851766286,3.650000


In [21]:
from surprise import SVD

In [22]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8701  0.8657  0.8720  0.8714  0.8705  0.8699  0.0022  
Fit time          1.25    1.29    1.28    1.14    1.12    1.22    0.07    
Test time         0.14    0.26    0.14    0.13    0.23    0.18    0.05    


{'test_rmse': array([0.87005613, 0.8657259 , 0.87200107, 0.87144659, 0.87049232]),
 'fit_time': (1.2502989768981934,
  1.2946622371673584,
  1.2772407531738281,
  1.1379508972167969,
  1.1154749393463135),
 'test_time': (0.13895606994628906,
  0.25777506828308105,
  0.13672685623168945,
  0.12746000289916992,
  0.23069190979003906)}

In [23]:
trainset = data.build_full_trainset()

In [24]:
svd.fit(trainset)